# Mini Detection
This mini detection algorithm allows you to visually fine-tune detection parameters for each recording

**What you need:** 
1. Each recording consists of one folder with all of the sweeps (.mat files)
2. For blind detection, each recording's condition should be recorded on the metadata csv (see Github for template)

**What you'll get:**
1. A JSON file of all the parameters you used for each cell for future reference
2. The avg mini amplitude per cell recording
3. The mini Hz per cell recording
4. csvs separated by condition and also combined with the information from 2 and 3

# Parameter Tuning

In [ ]:
# Import tuner
import param_tune
from param_tune import ParameterTuner

# Specify folder with data and the mode (EPSC or IPSC)
parent_folder = ''  # Update this path
tuner = ParameterTuner(parent_folder, mode='EPSC')

# Define parameter sets to test
# Format: (window_size, sample_step, interval)
parameter_sets = [
    (150, 50, 25),  # Default EPSC parameters
     (125, 25, 15),   # Smaller windows, more sensitive
    # (300, 50, 70),  # Larger windows, less sensitive
    (125, 35, 20)   # fast sensitivity
]

# Test parameters on a specific cell
cell_name = "cell_name"   # Replace with actual cell name
results = tuner.sample_and_test(cell_name, parameter_sets, n_files=3, visualize=True)

In [ ]:
# Save optimal parameters for cells after testing
tuner.save_parameters("cell_name", window_size=150, sample_step=50, interval=25)

# Specify output dir
output_dir = ""

# Export parameters to JSON file for later use
tuner.export_parameters("optimized_parameters.json", output_dir)

# Detection

In [ ]:
# Run full detection with optimized parameters
optimized_results = tuner.run_full_detection(output_dir = output_dir)

# Group & Export Data

In [ ]:
# Import metadata file
import pandas as pd

metadata = pd.read_csv('metadata.csv')

# Examine metadata structure
print("Metadata columns:", metadata.columns.tolist())
print("\nFirst few rows of metadata:")
print(metadata.head())
print("\nUnique values in each column:")
for col in metadata.columns:
    print(f"{col}: {metadata[col].unique()}")
print(f"\nMetadata shape: {metadata.shape}")

In [ ]:
import functions

# Separate data by groups defined by metadata
grouped_data, ungrouped = functions.separate_data_by_groups(optimized_results, metadata, 
                                                 cell_id_col='Cell', 
                                                 group_col='Condition')

In [ ]:
# Export grouped data to separate CSV files

from functions import export_data, avg_amp_per_cell, hz_per_cell

# Specify date
date = '2025XXXX'

# Ensure output_dir has trailing slash
if not output_dir.endswith('/'):
    output_dir = output_dir + '/'

# Prepare data for export
condition_labels = []
amplitude_data = []
frequency_data = []
detection_data_list = []

for group_name, group_results in grouped_data.items():
    condition_labels.append(group_name)
    
    # Calculate amplitudes and frequencies for this group
    group_amplitudes = avg_amp_per_cell(group_results)
    group_frequencies = hz_per_cell(group_results)
    
    amplitude_data.append(group_amplitudes)
    frequency_data.append(group_frequencies)
    detection_data_list.append(group_results)

# Export amplitudes
export_data(
    condition_labels=condition_labels,
    big_data=amplitude_data,
    data_type='Amplitude',
    date=date,
    output_dir=output_dir,
    detected_data_list=detection_data_list
)

# Export frequencies
export_data(
    condition_labels=condition_labels,
    big_data=frequency_data,
    data_type='Frequency',
    date=date,
    output_dir=output_dir,
    detected_data_list=detection_data_list
)

print("\n=== Export Summary ===")
print(f"Exported data for {len(condition_labels)} groups:")
for i, label in enumerate(condition_labels):
    print(f"  {label}: {len(amplitude_data[i])} cells")
print(f"\nFiles saved to: {output_dir}")
print("Created files:")
print("  - Individual group CSV files for amplitudes and frequencies")
print("  - Combined CSV files with all groups together")